In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
import os.path
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
import scipy
Image.MAX_IMAGE_PIXELS = None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Veri seti klasörünün yolu (Data klasörünüzün yolu)
data_dir = Path('/content/drive/MyDrive/Train')
filepaths = list(data_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2,
    vertical_flip=False,
    rotation_range=30,    # Örneğin, 0 ile 90 derece arasında rastgele döndürme
    zoom_range=[0.5,1.5], # Rastgele yakınlaştırma aralığı
    width_shift_range=0.05,
    height_shift_range=0.1,
    brightness_range=[0.7,1],
    shear_range=10.0,
    channel_shift_range=80.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    rescale=None,
    data_format=None,
    dtype=None

) # %70 eğitim datası içinden ön eğitim kontrölü içinde %20 yani 140 adet ön test (daha önceden gördüğü veri üzerinden test yapmak için) veri ayırıyoruz

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,

    vertical_flip=False,
    rotation_range=30,    # Örneğin, 0 ile 90 derece arasında rastgele döndürme
    zoom_range=[0.5,1.5], # Rastgele yakınlaştırma aralığı
    width_shift_range=0.05,
    height_shift_range=0.1,
    brightness_range=[0.7,1],
    shear_range=10.0,
    channel_shift_range=80.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    rescale=None,
    data_format=None,
    dtype=None
    )


In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(256,256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=1024,
    subset='training'
)
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(256,256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=1024,
    subset='validation'
)
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(256,256),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)


Found 6500 validated image filenames belonging to 2 classes.
Found 1625 validated image filenames belonging to 2 classes.
Found 2032 validated image filenames belonging to 2 classes.


In [ ]:
pretrained_model = tf.keras.applications.resnet50.ResNet50(
                    input_shape=(256,256,3), # Resimde 3 girişli bir kanal (ağ) yapısı vardır standart ölçü
                    include_top=False,       # Katmanın başlangıç noktası (en üst kısmıdır) veri kalitesi ve eğitim açısından eklenmez bu yüzden False ise eklenmeyecek ise tanımlamak zorundayız
                    weights='imagenet',      # Fine-Tuning (öncedene eğitilmiş bir modeli alıyoruz)
                    pooling='avg')           # Girdi hacminin uzamsal boyutlarını azaltarak modelin hesaplama karmaşıklığını düşürümek için tanımlarız

pretrained_model.trainable = False

inputs = pretrained_model.input
x = tf.keras.layers.Dense(256, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(64, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)  # Örneğin, 0.5 olasılıkla Dropout uygula overfitting olması durumunda bu katmanı açacağız

outputs = tf.keras.layers.Dense(4, activation='softmax')(x) # 4 sınıf sayısıdır softmax (sigmoid ise max 2 sınıflı yapılar için kullanılır)
model = tf.keras.Model(inputs, outputs)

# Kendi özel CNN modelinizi oluşturun
# custom_model = models.Sequential([
#     resnet_base,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(256, activation='relu'),
#     layers.Dropout(0.5),
#    layers.Dense(4, activation='softmax')  # 4 sınıflı sınıflandırma için softmax aktivasyonu
#])


94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
adam=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Modeli en iyi noktasına ulaşıtığında kaydedetmek için
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
    filepath="model_data_augmennt_full_train_checkpoints",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch"
)

In [ ]:
# Sıkıntılı artış azalış durumlarında overfitting-underfitting görülür veri fazla ise overfitting az ise underfitting olmasını takip etmek ve en iyi öğrendiği nokktayı tespit edip
# durdurmak için Callback hyperparametereleri kullanılır böylece ideal öğrenme tespit edildiği noktada epoch sayısına bakmaksızın sistem durur ve model kaydedilir.
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True) # ayrıntı çubuğu için
history = model.fit(train_images,validation_data=val_images,epochs=100,batch_size =32,callbacks=[callbacks])



In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.savefig("8 Aralık/Val_Accury_bacth32_epoch100.png",dpi=200)
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.savefig("8 Aralık/Val_Loss_bacth32_epoch100.png",dpi=200)
plt.show()

In [ ]:
results = model.evaluate(test_images, verbose=0)
print(results)
print(f"Test Accuracy: {np.round(results[1] * 100,2)}%")

In [ ]:
predictions = np.argmax(model.predict(test_images), axis=1)
matrix = confusion_matrix(test_images.labels, predictions)
report= classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

In [ ]:
fig = plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, cmap='viridis')
plt.xticks(ticks=np.arange(4) + 0.5, labels=test_images.class_indices.keys(), rotation=90)
plt.yticks(ticks=np.arange(4) + 0.5, labels=test_images.class_indices.keys(), rotation=0)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
fig.savefig("8 Aralık/Confusion Matrix_Softmax_batch32_epoch100.png",dpi=200)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

# Modelinizden tahminler yapın
predictions = model.predict(test_images)

# Etiketleri ikili forma çevirin (one-hot encoding)
test_labels_bin = label_binarize(test_images.labels, classes=[0, 1, 2, 3])

# ROC eğrisini çıkarmak için One-vs-Rest sınıflandırıcı kullanın
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(test_images.class_indices)):
    fpr[i], tpr[i], _ = roc_curve(test_labels_bin[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Micro average ROC curve ve ROC alanını hesaplayın
fpr["micro"], tpr["micro"], _ = roc_curve(test_labels_bin.ravel(), predictions.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# ROC eğrisini çizin
plt.figure(figsize=(8, 6))

for i in range(len(test_images.class_indices)):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot(fpr["micro"], tpr["micro"], label=f'Micro Average (AUC = {roc_auc["micro"]:.2f})', linestyle='--', linewidth=2, color='black')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', linewidth=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.savefig("8 Aralık/ROC_bacth32_epoch100.png",dpi=200)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# Modelinizden tahminler yapın
predictions = model.predict(test_images)

# Etiketleri ikili forma çevirin (one-hot encoding)
test_labels_bin = label_binarize(test_images.labels, classes=[0, 1, 2, 3])

# ROC eğrisini çıkarmak için One-vs-Rest sınıflandırıcı kullanın
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(test_images.class_indices)):
    fpr[i], tpr[i], _ = roc_curve(test_labels_bin[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Micro average ROC curve ve ROC alanını hesaplayın
fpr["micro"], tpr["micro"], _ = roc_curve(test_labels_bin.ravel(), predictions.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# ROC Curve ve Confusion Matrix'i aynı grafikte göster
plt.figure(figsize=(16, 6))


# Confusion Matrix
plt.subplot(1, 2, 1)

# Örnek confusion matrix
cm = confusion_matrix(test_images.labels, np.argmax(predictions, axis=1))

# Görselleştirme
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
classes = list(test_images.class_indices.keys())
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.tight_layout()
plt.savefig("8 Aralık/Confusion Matrix_bacth32_epoch100.png",dpi=200)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# Modelinizden tahminler yapın
predictions = model.predict(test_images)

# Etiketleri ikili forma çevirin (one-hot encoding)
test_labels_bin = label_binarize(test_images.labels, classes=[0, 1, 2, 3])

# Precision-Recall Curve'ü çıkarmak için One-vs-Rest sınıflandırıcı kullanın
precision = dict()
recall = dict()
average_precision = dict()

for i in range(len(test_images.class_indices)):
    precision[i], recall[i], _ = precision_recall_curve(test_labels_bin[:, i], predictions[:, i])
    average_precision[i] = average_precision_score(test_labels_bin[:, i], predictions[:, i])

# Micro average Precision-Recall curve ve Precision alanını hesaplayın
precision["micro"], recall["micro"], _ = precision_recall_curve(test_labels_bin.ravel(), predictions.ravel())
average_precision["micro"] = average_precision_score(test_labels_bin, predictions, average="micro")

# Precision-Recall Curve,  grafikte göster
plt.figure(figsize=(12, 6))

# Precision-Recall Curve
plt.subplot(1, 3, 1)

for i in range(len(test_images.class_indices)):
    plt.plot(recall[i], precision[i], label=f'Class {i} (AP = {average_precision[i]:.2f})')

plt.plot(recall["micro"], precision["micro"], label=f'Micro Average (AP = {average_precision["micro"]:.2f})', linestyle='--', linewidth=2, color='black')

plt.xlabel('Duyarlılık (Recall)')
plt.ylabel('Hassasiyet (Precision)')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower right')
plt.savefig("8 Aralık/Precision-Recall Curve_bacth32_epoch100.png",dpi=200)

plt.tight_layout()
plt.show()


In [ ]:
print("Classification Report:\n", report)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
model.save('8 Aralık/ResNet_bacth32_epoch100_Softmax_Drop.h5')